<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao20_Comparacao_TFIDF_vs_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings

# Suprimir avisos para um output mais limpo
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

# --- 1. PREPARAÇÃO DOS DADOS ---
# Vamos usar um subconjunto do famoso dataset "20 Newsgroups"
# Nosso objetivo será classificar se um post é sobre computação gráfica ou criptografia
categorias = ['comp.graphics', 'sci.crypt']
dados = fetch_20newsgroups(subset='all', categories=categorias, shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))

# Separar dados e rótulos
textos = dados.data
rotulos = dados.target # 0 para 'comp.graphics', 1 para 'sci.crypt'

# Dividir em treino e teste
X_train_text, X_test_text, y_train, y_test = train_test_split(textos, rotulos, test_size=0.2, random_state=42)

print(f"Total de amostras: {len(textos)}")
print(f"Amostras de treino: {len(X_train_text)}")
print(f"Amostras de teste: {len(X_test_text)}")
print(f"Rótulo de exemplo (0='comp.graphics', 1='sci.crypt'): {y_train[0]}")
print(f"Texto de exemplo: \n{X_train_text[0][:200]}...\n")

# --- 2. ABORDAGEM 1: TF-IDF + Regressão Logística (Machine Learning Clássico) ---
print("\n--- INICIANDO MODELO 1: TF-IDF + Regressão Logística ---")

# Criar o vetorizador TF-IDF
# max_features=5000 -> usar apenas as 5000 palavras mais comuns
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

# Aprender o vocabulário e transformar os dados de treino
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)

# Apenas transformar os dados de teste (usando o vocabulário de treino)
X_test_tfidf = tfidf_vectorizer.transform(X_test_text)

print(f"Formato dos dados de treino (TF-IDF): {X_train_tfidf.shape}")
print(f"Formato dos dados de teste (TF-IDF): {X_test_tfidf.shape}")

# Treinar um classificador simples
modelo_logistico = LogisticRegression()
modelo_logistico.fit(X_train_tfidf, y_train)

# Fazer previsões e avaliar
predicoes_tfidf = modelo_logistico.predict(X_test_tfidf)
acuracia_tfidf = accuracy_score(y_test, predicoes_tfidf)

print(f"Acurácia (TF-IDF + Regressão Logística): {acuracia_tfidf:.4f}")
print("--- MODELO 1 CONCLUÍDO ---\n")


# --- 3. ABORDAGEM 2: Embeddings + RNN (LSTM) (Deep Learning) ---
print("\n--- INICIANDO MODELO 2: Embedding + RNN (LSTM) ---")

# Parâmetros para a RNN
MAX_PALAVRAS_VOCAB = 10000 # Tamanho máximo do vocabulário (diferente do TF-IDF)
MAX_COMPRIMENTO_SEQ = 250  # Comprimento máximo de cada documento (em palavras)
DIM_EMBEDDING = 100        # Dimensão de cada vetor de palavra (embedding)

# 1. Tokenizar o texto (converter palavras em números inteiros)
tokenizer = Tokenizer(num_words=MAX_PALAVRAS_VOCAB, oov_token="<OOV>") # <OOV> para palavras fora do vocabulário
tokenizer.fit_on_texts(X_train_text)

# Converter textos em sequências de inteiros
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# 2. Padronizar as sequências (para que todas tenham o mesmo comprimento)
# RNNs exigem entradas de comprimento fixo
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_COMPRIMENTO_SEQ, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_COMPRIMENTO_SEQ, padding='post', truncating='post')

print(f"Formato dos dados de treino (Padronizados): {X_train_pad.shape}")
print(f"Formato dos dados de teste (Padronizados): {X_test_pad.shape}")
print(f"Exemplo de sequência padronizada: \n{X_train_pad[0][:20]}...")

# 3. Construir o modelo RNN (usando LSTM)
modelo_rnn = Sequential(name="Modelo_RNN_LSTM")

# Camada de Embedding: Transforma os inteiros (índices de palavras) em vetores densos
# Ela aprende o "significado" das palavras durante o treino
modelo_rnn.add(Embedding(input_dim=MAX_PALAVRAS_VOCAB,
                         output_dim=DIM_EMBEDDING,
                         input_length=MAX_COMPRIMENTO_SEQ))

# Dropout para regularização
modelo_rnn.add(SpatialDropout1D(0.2))

# A camada Recorrente (LSTM) que processa a sequência
modelo_rnn.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))

modelo_rnn.add(Dense(16, activation='relu'))

# Camada de saída: 1 neurônio com 'sigmoid' para classificação binária (0 ou 1)
modelo_rnn.add(Dense(1, activation='sigmoid'))

# Compilar o modelo
modelo_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("\nEstrutura do Modelo RNN:")
modelo_rnn.summary()

# 4. Treinar o modelo RNN
# (Usamos os rótulos y_train/y_test diretamente)
# Convertendo rótulos para numpy array para garantir compatibilidade
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

print("\nTreinando o modelo RNN...")
historico = modelo_rnn.fit(X_train_pad, y_train_np,
                           epochs=100,
                           batch_size=32,
                           validation_data=(X_test_pad, y_test_np),
                           verbose=2) # verbose=2 mostra uma linha por época

# 5. Avaliar o modelo
loss, acuracia_rnn = modelo_rnn.evaluate(X_test_pad, y_test_np, verbose=0)

print(f"\nAcurácia (Embedding + RNN): {acuracia_rnn:.4f}")
print("--- MODELO 2 CONCLUÍDO ---")

# --- 4. COMPARAÇÃO FINAL ---
print("\n\n--- COMPARAÇÃO DOS RESULTADOS ---")
print(f"Acurácia (TF-IDF + Logística): {acuracia_tfidf:.4f}")
print(f"Acurácia (Embedding + RNN):    {acuracia_rnn:.4f}")
print("---------------------------------")
print("Observe como as duas abordagens exigem pré-processamentos de dados totalmente diferentes.")
print("TF-IDF cria vetores 'largos' e esparsos (bag-of-words).")
print("Embedding/RNN cria sequências 'longas' de vetores densos (sequencial).")

Total de amostras: 1964
Amostras de treino: 1571
Amostras de teste: 393
Rótulo de exemplo (0='comp.graphics', 1='sci.crypt'): 1
Texto de exemplo: 
...


--- INICIANDO MODELO 1: TF-IDF + Regressão Logística ---
Formato dos dados de treino (TF-IDF): (1571, 5000)
Formato dos dados de teste (TF-IDF): (393, 5000)
Acurácia (TF-IDF + Regressão Logística): 0.9160
--- MODELO 1 CONCLUÍDO ---


--- INICIANDO MODELO 2: Embedding + RNN (LSTM) ---
Formato dos dados de treino (Padronizados): (1571, 250)
Formato dos dados de teste (Padronizados): (393, 250)
Exemplo de sequência padronizada: 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]...

Estrutura do Modelo RNN:


Model: "Modelo_RNN_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando o modelo RNN...
Epoch 1/100
50/50 - 22s - 435ms/step - accuracy: 0.5226 - loss: 0.6936 - val_accuracy: 0.5445 - val_loss: 0.6903
Epoch 2/100
50/50 - 20s - 404ms/step - accuracy: 0.5328 - loss: 0.6891 - val_accuracy: 0.5471 - val_loss: 0.6827
Epoch 3/100
50/50 - 15s - 305ms/step - accuracy: 0.5411 - loss: 0.6740 - val_accuracy: 0.5471 - val_loss: 0.6804
Epoch 4/100
50/50 - 15s - 303ms/step - accuracy: 0.5608 - loss: 0.6560 - val_accuracy: 0.5496 - val_loss: 0.6877
Epoch 5/100
50/50 - 21s - 413ms/step - accuracy: 0.5888 - loss: 0.6399 - val_accuracy: 0.5547 - val_loss: 0.6759
Epoch 6/100
50/50 - 15s - 308ms/step - accuracy: 0.5913 - loss: 0.6317 - val_accuracy: 0.5598 - val_loss: 0.6758
Epoch 7/100
50/50 - 16s - 322ms/step - accuracy: 0.5964 - loss: 0.6068 - val_accuracy: 0.5802 - val_loss: 0.6384
Epoch 8/100
50/50 - 20s - 404ms/step - accuracy: 0.5945 - loss: 0.6040 - val_accuracy: 0.5547 - val_loss: 0.6909
Epoch 9/100
50/50 - 20s - 409ms/step - accuracy: 0.5913 - loss: 0.592